In [ ]:
import logging
from datetime import datetime
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from _spotify.spotipy_auth import spotipy_auth
from _auth.get_project_id import get_project_id
from _auth.google_auth import google_auth

def categories():
    try:
        spotipy_auth()
        spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
        result = spotify.categories(limit = 50, country = 'US')
        df = pd.DataFrame(result['categories']['items'])
        timestamp = datetime.now()
        df['date'] = timestamp
        google_auth()
        df.to_csv(
            f"gs://{get_project_id()}"
            f"/spotify/global_top_50/{timestamp.strftime('%Y-%m-%d')}.csv"
        )
    except Exception as e:
        logging.error(e)

categories()

In [ ]:
def query_new_releases():
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

    result = spotify.new_releases(country = 'US', limit = 50)

    df = pd.DataFrame(result['albums']['items'])
    df = df.copy()
    df['artists_name'] = df['artists'].apply(lambda row: [artist['name'] for artist in row])
    df['artists_id'] = df['artists'].apply(lambda row: [artist['id'] for artist in row])
    
    return df[['album_type', 
               'artists_name', 
               'name', 
               'release_date', 
               'total_tracks', 
               'id',
               'artists_id']]

query_new_releases().head()

In [ ]:
artist_id = "53KwLdlmrlCelAZMaLVZqU"

def query_artist(artist_id):
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

    result = spotify.artist_related_artists(artist_id)

    df = pd.DataFrame(result['artists'])

    df = df[['name',
             'followers',
             'genres',
             'popularity',
             'id']]

    df = df.copy()
    df['followers'] = df['followers'].apply(lambda row: row['total'])
    return df

query_artist(artist_id).head()

In [ ]:
def query_top_playlists():
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    result = spotify.category_playlists(category_id = 'toplists', limit = 50, country = 'US')
    df = pd.DataFrame(result['playlists']['items'])
    return df[['name', 'description', 'id']]

query_top_playlists()

In [ ]:
def query_top_50_global():
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

    result = spotify.playlist_tracks(playlist_id = '37i9dQZEVXbMDoHDwVN2tF', limit = 50)
    
    df = pd.DataFrame(result['items'])
    df = df[['added_at',
             'track']]

    df = df.copy()
    df['track_id'] = df['track'].apply(lambda row: row['id'])

    df['track_name'] = df['track'].apply(lambda row: row['name'])
    df['artists_id'] = df['track'].apply(lambda row: [artist['id'] for artist in row['artists']])
    df['artists_name'] = df['track'].apply(lambda row: [artist['name'] for artist in row['artists']])

    return df[['track_name', 
              'artists_name', 
              'added_at',
              'track_id', 
              'artists_id']]

query_top_50_global().head()